This notebook contains evaluation of LongT5 Model trained on pubmed research paper dataset. ROUGE is used for measuring accuracy. The test example used is from BigPatent dataset.
https://huggingface.co/whaleloops/longt5-tglobal-large-16384-pubmed-10k_steps

This notebook has not been executed so scores are not mentioned.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install rouge-score

In [3]:
from datasets import load_dataset

ds = load_dataset("pszemraj/scientific_lay_summarisation-elife-norm")


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#import pandas as pd
# put data into dataFrames and csv
#df_train = pd.DataFrame(ds['train'])
#df_val = pd.DataFrame(ds['validation'])
#df_test = pd.DataFrame(ds['test'])
#df_train.to_csv("train.csv",index=False)
#df_val.to_csv("val.csv",index=False)
#df_test.to_csv("test.csv",index=False)

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title', 'article_length', 'summary_length'],
        num_rows: 4346
    })
    validation: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title', 'article_length', 'summary_length'],
        num_rows: 241
    })
    test: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title', 'article_length', 'summary_length'],
        num_rows: 241
    })
})

In [6]:
# cleaning text
import re

# def remove_sentence_with_fig(paragraph):


#     # first we remove . after FIG
#     paragraph = re.sub(r"FIG.","FIG",paragraph)

#     # Split the paragraph into sentences
#     sentences = re.split(r'(?<=[.!?])\s+', paragraph)

#     # Remove sentences containing "FIG."
#     filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

#     # Join the remaining sentences to form the updated paragraph
#     updated_paragraph = ' '.join(filtered_sentences)

#     return updated_paragraph

def clean_text(text):

    # replace ° degree with
    text = re.sub(r"°", " degree", text)

    # remove et al.
    text = re.sub(r'\bet al\.\b', '', text)

    text = re.sub(r"FIG.","FIG", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    text = ' '.join(filtered_sentences)


    text = text.strip()

    return text


sections = ds['test']['article'][0]
print(sections,'\n')
text = "".join(["".join(sentence) for sentence in sections])



cleaned_text = clean_text(text)
# text = remove_sentence_with_fig(cleaned_text)

# print(cleaned_text)
print(cleaned_text)

Most movements are not unitary, but are comprised of sequences. Although patients with cerebellar pathology display severe deficits in the execution and learning of sequences (Doyon et al. , 1997; Shin and Ivry, 2003), most of our understanding of cerebellar mechanisms has come from analyses of single component movements. Eyelid conditioning is a cerebellar-mediated behavior that provides the ability to control and restrict inputs to the cerebellum through stimulation of mossy fibers. We utilized this advantage to test directly how the cerebellum can learn a sequence of inter-connected movement components in rabbits. We show that the feedback signals from one component are sufficient to serve as a cue for the next component in the sequence. In vivo recordings from Purkinje cells demonstrated that all components of the sequence were encoded similarly by cerebellar cortex. These results provide a simple yet general framework for how the cerebellum can use simple associate learning proces

In [7]:
 # divide the text into paragraphs of 16000
text1 = cleaned_text.split()
text_lst = []
for i in range(0,len(text1),16000):
  tmp = " ".join(text1[i:i+16000])
  text_lst.append(tmp)


In [8]:
# exploring the dataset

print(ds['test']['summary'][0])

Imagine a gymnastics competition in which participants take turns to cartwheel and somersault across the floor. The routines on display comprise sequences of precisely timed movements learned through practice. This is also true for many of the actions we perform every day, such as reaching for a cup of coffee. A region of the brain called the cerebellum helps us learn sequences of movements. But how does it do this? To find out, Khilkevich et al. came up with a new version of an old experiment. Rabbits were first trained to blink their eye in response to a specific external cue. This type of learning, called associative learning, has been shown before in the cerebellum. But Khilkevich et al. wondered whether the cerebellum could also use internal feedback signals from the eyeblink as a cue to learn the next movement? If so, this might explain how the cerebellum can chain movements together in a sequence. As predicted, Khilkevich et al. found that rabbits could learn to blink their eye 

In [9]:
# For cleaning predicted text
def clean_predicted_text(text):


    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)

    #replace new line character
    text = re.sub(r"<n>", "\n", text)

    #replace start of paragraph
    text = re.sub(r"<s>", "", text)

    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)




    return text

In [10]:
# creating model
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model/")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model/")
# summary expected length
#expected_length = len(ds['test'][0]["abstract"])//2

summarizer = pipeline("summarization",model=model,tokenizer=tokenizer,num_beams=4,no_repeat_ngram_size=4,top_k=20)


In [11]:
# checkpoint = "EleutherAI/pythia-1.4b-deduped"
# assistant_checkpoint = "EleutherAI/pythia-160m-deduped"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# inputs = tokenizer(prompt, return_tensors="pt")

# model = AutoModelForCausalLM.from_pretrained(checkpoint)
# assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint)
# outputs = model.generate(**inputs, assistant_model=assistant_model, do_sample=True, temperature=0.5)
# tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [12]:

#summarize data
#pred_summary_length = expected_length//len(text_lst)
predictions = []
for i in range(0,len(text_lst)):
  tmp = text_lst[i]

  prediction = clean_predicted_text(summarizer(tmp,min_length=200,max_length=500,do_sample=True)[0]["summary_text"])
  #predictions
  predictions.append(prediction)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [13]:
from rouge_score import rouge_scorer
abstract = ds['test']['summary'][0]
predicted_abstract = " ".join(predictions)
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(abstract, predicted_abstract)

In [14]:
predictions

['We show that the cerebellum can learn to chain together a sequence of inter-connected movement components by using feedback signals from one component to serve as a cue for the next component in the sequence Figure 1C, first movement. In vivo recordings from cerebellar cortex showed that Purkinje cell activity relates as strongly to the latter components of sequences as it has been shown to relate to single-component eyelid CRs Halverson et al. In this study, we show that a simple elaboration of cerebellar mechanisms that mediate learning of movement sequences is sufficient to explain cerebellar learning and implementation of movements sequences, and provide a general framework for how the cerebellal can use feedback signals to learn to chain within an appropriately timed response to produce a movement sequence. We show that mossy fiber stimulation CS is sufficient to drive the second CR in ipsilateral sequence, since the CS was delivered through electrodes implanted in the left midd

In [15]:
#rouge max_length tokens 100
scores


{'rouge1': Score(precision=0.48404255319148937, recall=0.5, fmeasure=0.4918918918918919),
 'rouge2': Score(precision=0.14133333333333334, recall=0.14600550964187328, fmeasure=0.14363143631436315),
 'rougeL': Score(precision=0.21010638297872342, recall=0.21703296703296704, fmeasure=0.21351351351351353)}

In [16]:
abstract

'Imagine a gymnastics competition in which participants take turns to cartwheel and somersault across the floor. The routines on display comprise sequences of precisely timed movements learned through practice. This is also true for many of the actions we perform every day, such as reaching for a cup of coffee. A region of the brain called the cerebellum helps us learn sequences of movements. But how does it do this? To find out, Khilkevich et al. came up with a new version of an old experiment. Rabbits were first trained to blink their eye in response to a specific external cue. This type of learning, called associative learning, has been shown before in the cerebellum. But Khilkevich et al. wondered whether the cerebellum could also use internal feedback signals from the eyeblink as a cue to learn the next movement? If so, this might explain how the cerebellum can chain movements together in a sequence. As predicted, Khilkevich et al. found that rabbits could learn to blink their eye